In [1]:
#All the imports in one place

import time
import datetime

import cPickle as pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pylab
import re
import scipy as sp
import seaborn

from gensim import corpora, models
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.qda import QDA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import *
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, roc_auc_score
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter
import os
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")


plt.rc('figure', figsize=(10,6))
seaborn.set()
colors = seaborn.color_palette()




C:\Users\Karan\Anaconda2\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Karan\Anaconda2\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
C:\Users\Karan\Anaconda2\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\Karan\Anaconda2\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [2]:
# Function to create the LDA model from the training dataset.
def perform_lda(train, totalTopics):
    corpus = []
    for text in train:
        text_list = [word for word in text]
        try:
            corpus.append(text_list)
        except:
            pass

    # Build dictionary
    dictionary = corpora.Dictionary(corpus)
    # Save the dictionary
    dictionary.save('restaurant_reviews.dict')
        
    # Build vectorized corpus
    corpus_vector = [dictionary.doc2bow(text) for text in corpus]
    
    lda = models.LdaModel(corpus_vector, num_topics=totalTopics, id2word=dictionary)
    return lda

In [3]:
# Generates a matrix of topic probabilities for each document in matrix
# Returns topic_dist for the input corpus, and all_dist, a running sum of all the corpuses
def getTopicDistMatrix(lda, totalTopics, corpus, all_dist, star):
    topic_dist = [0] * totalTopics
    # Load the dictionary
    dictionary = corpora.Dictionary.load("restaurant_reviews.dict")
    # For every reviewDoc in the corpus, compute the prob dist matrix for each term
    for review in corpus:
        vec = dictionary.doc2bow(review.lower().split())
        output = lda[vec]
        highest_prob = 0
        highest_topic = 0
        temp = [0] * totalTopics    # List to keep track of topic distribution for each document
        for topic in output:
            this_topic, this_prob = topic
            temp[this_topic] = this_prob
            if this_prob > highest_prob:
                highest_prob = this_prob 
                highest_topic = this_topic
        temp.append(star)
        all_dist.append(temp)
        topic_dist[highest_topic] += 1
    return topic_dist, all_dist

In [5]:
all_text_train = pickle.load (open("all_text_train.p", "rb"))

In [6]:
corpus_5stars_train = pickle.load (open("corpus_5stars_train.p", "rb"))
corpus_4stars_train = pickle.load (open("corpus_4stars_train.p", "rb"))
corpus_3stars_train = pickle.load (open("corpus_3stars_train.p", "rb"))
corpus_2stars_train = pickle.load (open("corpus_2stars_train.p", "rb"))
corpus_1stars_train = pickle.load (open("corpus_1stars_train.p", "rb"))

In [7]:
%time lda = perform_lda(all_text_train, 15)

Wall time: 16.9 s


In [8]:
topic_dist_list = []

# Keep a separate list to count topics
topic_dist_5stars = []
topic_dist_4stars = []
topic_dist_3stars = []
topic_dist_2stars = []
topic_dist_1stars = []

In [9]:
totalTopics = 15
topic_dist_5stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_5stars_train, topic_dist_list, 5)
topic_dist_4stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_4stars_train, topic_dist_list, 4)
topic_dist_3stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_3stars_train, topic_dist_list, 3)
topic_dist_2stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_2stars_train, topic_dist_list, 2)
topic_dist_1stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_1stars_train, topic_dist_list, 1)

In [10]:
cols = []
for i in xrange(1, totalTopics+1):
    cols.append("Topic"+ str(i))
cols.append("Star")


In [11]:
topic_dist_train_all_stars = pd.DataFrame(topic_dist_list, columns=cols)

In [12]:
all_text_test = pickle.load (open("all_text_test.p", "rb"))
corpus_5stars_test = pickle.load (open("corpus_5stars_test.p", "rb"))
corpus_4stars_test = pickle.load (open("corpus_4stars_test.p", "rb"))
corpus_3stars_test = pickle.load (open("corpus_3stars_test.p", "rb"))
corpus_2stars_test = pickle.load (open("corpus_2stars_test.p", "rb"))
corpus_1stars_test = pickle.load (open("corpus_1stars_test.p", "rb"))

starsGroup = pickle.load (open("starsGroup.p", "rb"))

In [14]:
topic_dist_list = []

# Keep a separate list to count topics
topic_dist_5stars = []
topic_dist_4stars = []
topic_dist_3stars = []
topic_dist_2stars = []
topic_dist_1stars = []


topic_dist_5stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_5stars_test, topic_dist_list, 5)
topic_dist_4stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_4stars_test, topic_dist_list, 4)
topic_dist_3stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_3stars_test, topic_dist_list, 3)
topic_dist_2stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_2stars_test, topic_dist_list, 2)
topic_dist_1stars, topic_dist_list = getTopicDistMatrix(lda, totalTopics, corpus_1stars_test, topic_dist_list, 1)

In [15]:
cols = []
for i in xrange(1, totalTopics+1):
    cols.append("Topic"+ str(i))
cols.append("Star")

In [16]:
topic_dist_test_all_stars = pd.DataFrame(topic_dist_list, columns=cols)

features = list(topic_dist_train_all_stars.columns[:totalTopics])

In [18]:
x_train = topic_dist_train_all_stars[features]
y_train = topic_dist_train_all_stars['Star']

x_test = topic_dist_test_all_stars[features]
y_test = topic_dist_test_all_stars['Star'] 

In [19]:
clfs = [MultinomialNB(), LogisticRegression(), RandomForestClassifier(n_estimators=100, n_jobs=2), AdaBoostClassifier(n_estimators=100)]
clf_names = ['Multinomial Naive Bayes', 'Logistic Regression', 'Random Forest', 'AdaBoost']

LDAResults = {}
for (i, clf_) in enumerate(clfs):
    clf = clf_.fit(x_train, y_train)
    preds = clf.predict(x_test)
    
    precision = metrics.precision_score(y_test, preds)
    recall = metrics.recall_score(y_test, preds)
    f1 = metrics.f1_score(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    report = classification_report(y_test, preds)
    matrix = metrics.confusion_matrix(y_test, preds, labels=starsGroup.groups.keys())
    
    data = {'precision':precision,
            'recall':recall,
            'f1_score':f1,
            'accuracy':accuracy,
            'clf_report':report,
            'clf_matrix':matrix,
            'y_predicted':preds}
    
    LDAResults[clf_names[i]] = data

cols = ['precision', 'recall', 'f1_score', 'accuracy']
LDA_Prediction_Perf = pd.DataFrame(LDAResults).T[cols].T
LDA_Prediction_Perf

C:\Users\Karan\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1203: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
C:\Users\Karan\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Karan\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1304: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_la

,AdaBoost,Logistic Regression,Multinomial Naive Bayes,Nearest Neighbors,Random Forest
precision,0.201363,0.201363,0.201363,0.201363,0.201363
recall,0.448735,0.448735,0.448735,0.448735,0.448735
f1_score,0.277985,0.277985,0.277985,0.277985,0.277985
accuracy,0.448735,0.448735,0.448735,0.448735,0.448735


In [20]:
sys.stdout = open("lda_results.txt", 'a')

In [22]:
for model, val in LDAResults.iteritems():
    print ('-------'+'-'*len(model))
    print 'MODEL:', model
    print ('-------'+'-'*len(model))
    print ('The precision for this classifier is ' + str(val['precision']))
    print ('The recall for this classifier is    ' + str(val['recall']))
    print ('The f1 for this classifier is        ' + str(val['f1_score']))
    print ('The accuracy for this classifier is  ' + str(val['accuracy']))
    print ('The confusion matrix for this classifier is  \n' + str(val['clf_matrix']))